In [7]:
import pandas as pd
import numpy as np

# Load the messy data
df = pd.read_csv('Data/marketing_campaign_data_messy.csv')

print(f"Loaded Dataset: {df.shape[0]} rows, {df.shape[1]} columns")

Loaded Dataset: 2020 rows, 12 columns


In [5]:
df

,Campaign_ID,Campaign_Name,Start_Date,End_Date,Channel,Impressions,Clicks,Spend,Conversions,Active,Clicks,Campaign_Tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24 00:00:00,2023-12-13,TikTok,16795,197,$102.82,20.0,Y,NaN,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06 00:00:00,2023-05-12,Facebook,1860,30,24.33,1.0,0,NaN,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13 00:00:00,2023-12-20,Email,77820,843,1323.39,51.0,No,NaN,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,2023-10-30,2023-11-03,TikTok,55886,2019,2180.38,135.0,True,NaN,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22 00:00:00,2023-04-23,Facebook,7265,169,252.44,30.0,Yes,NaN,FA
...,...,...,...,...,...,...,...,...,...,...,...,...
2015,CMP-00400,Q3_Summer_CMP-00400,2023-10-31 00:00:00,2023-11-13,TikTok,30592,586,$503.95,77.0,1,NaN,TI
2016,CMP-01255,Q4_Summer_CMP-01255,2023-09-01 00:00:00,2023-09-26,Google Ads,20097,897,1641.0,162.0,0,NaN,GO
2017,CMP-01050,Q2_Launch_CMP-01050,2023-02-09 00:00:00,2023-02-21,Instagram,33254,1117,883.82,214.0,0,NaN,IN
2018,CMP-01118,Q4_Winter_CMP-01118,2023-03-30 00:00:00,2023-04-27,Facebook,68728,2960,4198.5,591.0,Yes,NaN,FA


In [8]:
# STEPS:

# 1. Header Cleaning
# 2. Type conversion and currency cleaning
# 3. Categorical typos (fuzzy logic)
# 4. Handling mixed booleans
# 8. Handling Outliers (Winsorizing)
# 9. String parsing (Feature extraction)

In [8]:
# STEP 1:

print(df.columns.tolist()) # Search for anomalies like spaces or other just like " Campaign_ID " -> spaces = problem

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
print("FIX APPLIED")

print(df.columns.tolist()) # Final check

[' Campaign_ID ', 'Campaign_Name', 'Start_Date', 'End_Date', 'Channel', 'Impressions', 'Clicks ', 'Spend', 'Conversions', 'Active', 'Clicks', 'Campaign_Tag']
FIX APPLIED
['campaign_id', 'campaign_name', 'start_date', 'end_date', 'channel', 'impressions', 'clicks', 'spend', 'conversions', 'active', 'clicks', 'campaign_tag']


In [9]:
# STEP 2:

dirty_spend_mask = df['spend'].astype(str).str.contains(r'\$') # The problem here is that we have an $ (string) with a number (float/double)
print(df.loc[dirty_spend_mask,['campaign_id','spend']].head(3)) # Show us the problem in the first 3 rows and only campaign_id and spend columns

df['spend'] = df['spend'].astype(str).str.replace(r'[^\d.-]', '', regex=True) # Only let numbers now
df['spend'] = pd.to_numeric(df['spend']) # Convert to numeric
print("FIX APPLIED")

print(df.loc[dirty_spend_mask,['campaign_id','spend']].head(3)) # Final check

   campaign_id     spend
0    CMP-00001   $102.82
21   CMP-00022   $2428.4
22   CMP-00023  $4726.22
FIX APPLIED
   campaign_id    spend
0    CMP-00001   102.82
21   CMP-00022  2428.40
22   CMP-00023  4726.22


In [10]:
# STEP 3:

print(df['channel'].unique()) # Different values that refers to the same thing 'insta' == 'instagram' so should be only instagram

cleanup_map = {
    'Facebok': 'Facebook',
    'Insta_gram': 'Instagram',
    'Gogle': 'Google Ads',
    'Tik_Tok': 'TikTok',
    'E-mail': 'Email',
    'N/A': np.nan  # Handling the ghost value here too
}

df['channel'] = df['channel'].replace(cleanup_map)
print("FIX APPLIED")

print(df['channel'].unique()) # Final check

['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' 'E-mail' nan 'Gogle'
 'Tik_Tok' 'Facebok' 'Insta_gram']
FIX APPLIED
['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' nan]


In [11]:
# STEP 4:

print(df['active'].unique())

bool_map = {
    'Yes': True,
    'Y': True,
    '1': True,
    1: True,
    'No': False,
    'N': False,
    '0': False,
    0: False
}

df['active'] = df['active'].map(bool_map).fillna(False).infer_objects(copy=False).astype(bool) # Replace using the bool_map created above and any N/A value goes to False
print("FIX APPLIED")

print(df['active'].unique()) # Final check

['Y' '0' 'No' 'True' 'Yes' '1' 'False']
FIX APPLIED
[ True False]


C:\Users\matte\AppData\Local\Temp\ipykernel_2560\2028554945.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['active'] = df['active'].map(bool_map).fillna(False).infer_objects(copy=False).astype(bool) # Replace using the bool_map created above and any N/A value goes to False


In [12]:
# STEP 5:

print(df['start_date'].dtype) # start_type type is not datetype but object

df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
df['end_date'] = pd.to_datetime(df['end_date'], dayfirst=True, errors='coerce')
print("FIX APPLIED")

print(df['start_date'].dtype)

object
FIX APPLIED
datetime64[ns]


C:\Users\matte\AppData\Local\Temp\ipykernel_2560\3460467132.py:6: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['end_date'] = pd.to_datetime(df['end_date'], dayfirst=True, errors='coerce')


In [14]:
df = df.loc[:, ~df.columns.duplicated()] # locate duplicated columns and removes them
df

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,campaign_tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24,2023-12-13,TikTok,16795,197,102.82,20.0,True,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06,2023-05-12,Facebook,1860,30,24.33,1.0,False,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13,2023-12-20,Email,77820,843,1323.39,51.0,False,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,NaT,2023-11-03,TikTok,55886,2019,2180.38,135.0,False,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22,2023-04-23,Facebook,7265,169,252.44,30.0,True,FA
...,...,...,...,...,...,...,...,...,...,...,...
2015,CMP-00400,Q3_Summer_CMP-00400,2023-10-31,2023-11-13,TikTok,30592,586,503.95,77.0,True,TI
2016,CMP-01255,Q4_Summer_CMP-01255,2023-09-01,2023-09-26,Google Ads,20097,897,1641.00,162.0,False,GO
2017,CMP-01050,Q2_Launch_CMP-01050,2023-02-09,2023-02-21,Instagram,33254,1117,883.82,214.0,False,IN
2018,CMP-01118,Q4_Winter_CMP-01118,2023-03-30,2023-04-27,Facebook,68728,2960,4198.50,591.0,True,FA


In [15]:
# STEP 6:

impossible_mask = df['clicks'] > df['impressions']
print(df.loc[impossible_mask,['campaign_id','impressions','clicks']].head(3))

Empty DataFrame
Columns: [campaign_id, impressions, clicks]
Index: []


In [6]:
df

,Campaign_ID,Campaign_Name,Start_Date,End_Date,Channel,Impressions,Clicks,Spend,Conversions,Active,Clicks,Campaign_Tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24 00:00:00,2023-12-13,TikTok,16795,197,$102.82,20.0,Y,NaN,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06 00:00:00,2023-05-12,Facebook,1860,30,24.33,1.0,0,NaN,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13 00:00:00,2023-12-20,Email,77820,843,1323.39,51.0,No,NaN,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,2023-10-30,2023-11-03,TikTok,55886,2019,2180.38,135.0,True,NaN,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22 00:00:00,2023-04-23,Facebook,7265,169,252.44,30.0,Yes,NaN,FA
...,...,...,...,...,...,...,...,...,...,...,...,...
2015,CMP-00400,Q3_Summer_CMP-00400,2023-10-31 00:00:00,2023-11-13,TikTok,30592,586,$503.95,77.0,1,NaN,TI
2016,CMP-01255,Q4_Summer_CMP-01255,2023-09-01 00:00:00,2023-09-26,Google Ads,20097,897,1641.0,162.0,0,NaN,GO
2017,CMP-01050,Q2_Launch_CMP-01050,2023-02-09 00:00:00,2023-02-21,Instagram,33254,1117,883.82,214.0,0,NaN,IN
2018,CMP-01118,Q4_Winter_CMP-01118,2023-03-30 00:00:00,2023-04-27,Facebook,68728,2960,4198.5,591.0,Yes,NaN,FA


In [19]:
# STEP 7:

time_travel_mask = df['end_date'] < df['start_date'] #End date is lower than the start date
print(df.loc[time_travel_mask,['campaign_id','start_date','end_date']].head(3))

df.loc[time_travel_mask, 'end_date'] = df.loc[time_travel_mask, 'start_date'] + pd.Timedelta(days=30) # 30 day window between start and end dates
print("FIX APPLIED")

print(df.loc[time_travel_mask,['campaign_id','start_date','end_date']].head(3)) # Final check

Empty DataFrame
Columns: [campaign_id, start_date, end_date]
Index: []
FIX APPLIED
Empty DataFrame
Columns: [campaign_id, start_date, end_date]
Index: []


In [21]:
# STEP 8:

Q1 = df['spend'].quantile(.25)
Q3 = df['spend'].quantile(.75)

IQR = Q3 - Q1
upper_limit = Q3 + (3 * IQR)

outlier_mask = df['spend'] > upper_limit
print(df.loc[outlier_mask,['campaign_id', 'spend']].head(3))

print("FIX APPLIED")
df.loc[outlier_mask, 'spend'] = upper_limit
print(df.loc[outlier_mask,['campaign_id', 'spend']].head(3))

     campaign_id      spend
789    CMP-00790  500000.00
1443   CMP-01444    8921.51
1460   CMP-01461  500000.00
FIX APPLIED
     campaign_id      spend
789    CMP-00790  8603.5375
1443   CMP-01444  8603.5375
1460   CMP-01461  8603.5375


In [27]:
# STEP 9:

print(df['campaign_name'].head(3))

df['season'] = df['campaign_name'].str.extract(r'Q\d_([^_]+)_') # Extract only after the second _ getting the season

print("FIX APPLIED")

print(df[['campaign_name', 'season']].head(3))

0    Q4_Summer_CMP-00001
1    Q1_Launch_CMP-00002
2    Q3_Winter_CMP-00003
Name: campaign_name, dtype: object
FIX APPLIED
         campaign_name  season
0  Q4_Summer_CMP-00001  Summer
1  Q1_Launch_CMP-00002  Launch
2  Q3_Winter_CMP-00003  Winter
